# Lee-Carter Model

In [ ]:
import math
import numpy as np
import pmdarima as pm # Package used for the time series analysis and modeling

## Importing and arranging the historical data

In [ ]:
# Loading historical data for Switzerland
# Source: The Human Mortality Database. www.mortality.org
# Data is available from 1876 until 2020 for ages 0, 1, ..., 109, 110+
deathsPT = np.loadtxt("deathsPT.txt", skiprows = 3,
                      usecols = (2,3)) # 0th column: females; 1st column: males
exposuresPT = np.loadtxt("exposuresPT.txt", skiprows = 3,
                         usecols = (2,3)) # 0th column: females; 1st column: males
deathRatesFemalesPT = np.loadtxt("lifeTableFemalesPT.txt", skiprows = 3,
                                 usecols = (2))
deathRatesMalesPT = np.loadtxt("lifeTableMalesPT.txt", skiprows = 3,
                               usecols = (2))


# Defining relevant variables
periodAvailable = np.array([1876,2020]) # period for which data is available
agesAvailable = np.array([0,110]) # ages for which data is available
                                  # (category 110 captures all ages >= 110)

# PRE:  x in interval agesAvailable
#       t in interval periodAvailable
#       gender binary: 0 for female, 1 for male
# POST: Returns number of deaths at age x in year t for the respective gender
def dxt(x, t, gender):
    assert t <= periodAvailable[1] and t >= periodAvailable[0], f"Data not available for calendar year {t}!"
    deltaYears = t - periodAvailable[0]
    if x > agesAvailable[1]:
        print(f"For age {x}, only aggregate data including all ages greater and equal to {agesAvailable[1]} is available. This was used.")
        x = agesAvailable[1]
    assert x >= agesAvailable[0], f"Data not available for age {x}!"
    assert gender == 1 or gender == 0, "Gender must be binary!"
    return deathsPT[(agesAvailable[1] + 1) * deltaYears + x,gender]

# PRE:  x in interval agesAvailable
#       t in interval periodAvailable
#       gender binary: 0 for female, 1 for male
# POST: Returns exposure at age x in year t for the respective gender
def Ext(x, t, gender):
    assert t <= periodAvailable[1] and t >= periodAvailable[0], f"Data not available for calendar year {t}!"
    deltaYears = t - periodAvailable[0]
    if x > agesAvailable[1]:
        print(f"For age {x}, only aggregate data including all ages greater and equal to {agesAvailable[1]} is available. This was used.")
        x = agesAvailable[1]
    assert x >= agesAvailable[0], f"Data not available for age {x}!"
    assert gender == 1 or gender == 0, "Gender must be binary!"
    return exposuresPT[(agesAvailable[1] + 1) * deltaYears + x,gender]

# PRE:  x in interval agesAvailable
#       t in interval periodAvailable
#       gender binary: 0 for female, 1 for male
# POST: Returns central death rate at age x in year t for the respective gender
def mxt(x, t, gender):
    assert t <= periodAvailable[1] and t >= periodAvailable[0], f"Data not available for calendar year {t}!"
    deltaYears = t - periodAvailable[0]
    if x > agesAvailable[1]:
        print(f"For age {x}, only aggregate data including all ages greater and equal to {agesAvailable[1]} is available. This was used.")
        x = agesAvailable[1]
    assert x >= agesAvailable[0], f"Data not available for age {x}!"
    assert gender == 1 or gender == 0, "Gender must be binary!"
    if gender == 0:
        return deathRatesFemalesPT[(agesAvailable[1] + 1) * deltaYears + x]
    else:
        return deathRatesMalesPT[(agesAvailable[1] + 1) * deltaYears + x]

## Setting up all the functionalities of the model

In [ ]:
class Lee_Carter:
    
    # PRE:  ageStart <= ageEnd and both in interval agesAvailable
    #       periodStart <= periodEnd and both in interval periodAvailable
    #       gender binary: 0 for female, 1 for male
    # POST: Initializes a Lee-Carter model with the attributes given
    def __init__(self, ageStart, ageEnd, periodStart, periodEnd, gender, 
                 numOfYearsToForecast):
        if ageEnd > agesAvailable[1]:
            print(f"For ages greater than {agesAvailable[1]}, only aggregate data including all ages greater and equal to {agesAvailable[1]} is available. Hence, the maximum age was set to {agesAvailable[1]}.")
            ageEnd = agesAvailable[1]
        assert ageEnd >= ageStart, "The first age considered must be below or equal to the last age considered!"
        assert ageStart >= agesAvailable[0], f"Data not available for ages below {agesAvailable[0]}!"
        self.ages = np.arange(ageStart, ageEnd+1) # ages considered
        
        assert periodEnd >= periodStart, "The first year considered must be earlier than or the same as the last year considered!"
        assert periodEnd <= periodAvailable[1] and periodStart >= periodAvailable[0], "Data not available for the period selected!"
        self.years = np.arange(periodStart, periodEnd+1) # years considered
        
        assert gender == 1 or gender == 0, "Gender must be binary!"
        self.gender = gender # gender considered: 0 for female, 1 for male
        
        # estimated parameters alpha_x, beta_x and kappa_t
        self.alpha = np.zeros(ageEnd-ageStart+1)
        self.beta = np.ones(ageEnd-ageStart+1)
        self.kappa = np.zeros(periodEnd-periodStart+1)
        
        # approximate confidence intervals for alpha_x, beta_x and kappa_t
        # which will be obtained by bootstrapping
        self.alphaConfInt = np.zeros((ageEnd-ageStart+1, 2))
        self.betaConfInt = np.zeros((ageEnd-ageStart+1, 2))
        self.kappaConfInt = np.zeros((periodEnd-periodStart+1, 2))
        
        # fitted mortality and log-mortality rates
        # and approximate confidence intervals thereof
        # which will be obtained by bootstrapping
        self.logMortality = np.zeros((ageEnd-ageStart+1,
                                      periodEnd-periodStart+1))
        self.logMortConfInt = np.zeros((ageEnd-ageStart+1,
                                        periodEnd-periodStart+1, 2))
        self.mortality = np.zeros((ageEnd-ageStart+1,
                                   periodEnd-periodStart+1))
        self.mortConfInt = np.zeros((ageEnd-ageStart+1,
                                     periodEnd-periodStart+1, 2))
        
        # deviance residuals
        self.residuals = np.zeros((ageEnd-ageStart+1, periodEnd-periodStart+1))
        
        # Forecasts are produced for the following years
        self.yearsForecasted = np.arange(periodEnd+1,
                                         periodEnd+numOfYearsToForecast+1)
        
        # forecasts of kappa_t
        # and different types of confidence intervals thereof
        self.kappaForecast = np.zeros(numOfYearsToForecast)
        # confidence intervals corresponding to approach 1 in
        # subsection 2.3.1 of the bachelor thesis
        self.kappaConfIntForec = np.zeros((numOfYearsToForecast, 2))
        # confidence intervals corresponding to approach 2 in
        # subsection 2.3.1 of the bachelor thesis
        self.kappaAppConfIntForec = np.zeros((numOfYearsToForecast, 2))
        # confidence intervals corresponding to the combined approach in
        # subsection 2.3.1 of the bachelor thesis
        self.kappaCompConfIntForec = np.zeros((numOfYearsToForecast, 2))
        
        # forecasts of log-mortality rate
        # and different types of confidence intervals thereof
        self.logMortalityForecast = np.zeros((ageEnd-ageStart+1,
                                              numOfYearsToForecast))
        # corresponding to approach 1
        self.logMortConfIntForec = np.zeros((ageEnd-ageStart+1,
                                             numOfYearsToForecast, 2))
        # corresponding to approach 2
        self.logMortAppConfIntForec = np.zeros((ageEnd-ageStart+1,
                                                numOfYearsToForecast, 2))
        # corresponding to the combined approach
        self.logMortCompConfIntForec = np.zeros((ageEnd-ageStart+1,
                                                 numOfYearsToForecast, 2))
        
        # forecasts of mortality rate
        # and different types of confidence intervals thereof
        self.mortalityForecast = np.zeros((ageEnd-ageStart+1,
                                           numOfYearsToForecast))
        # corresponding to approach 1
        self.mortConfIntForec = np.zeros((ageEnd-ageStart+1,
                                          numOfYearsToForecast, 2))
        # corresponding to approach 2
        self.mortAppConfIntForec = np.zeros((ageEnd-ageStart+1,
                                             numOfYearsToForecast, 2))
        # corresponding to the combined approach
        self.mortCompConfIntForec = np.zeros((ageEnd-ageStart+1,
                                              numOfYearsToForecast, 2))
        
        self.fitted = False # Model already fitted?
        self.residualsComputed = False # Residuals already computed?
        self.bootstrapped = False # Bootstrapping already done?
        self.forecasted = False # Forecasts already produced?
        self.arima = None # fitted ARIMA model for time series of kappa_t
    
    # POST: Returns value of log-likelihood function without the constant term
    def getLogLikelihood(self):
        value = 0
        for t in self.years:
            for x in self.ages:
                value += dxt(x,t,self.gender) * (self.alpha[x-self.ages[0]] + self.beta[x-self.ages[0]]*self.kappa[t-self.years[0]])
                value -= Ext(x,t,self.gender) * np.exp(self.alpha[x-self.ages[0]] + self.beta[x-self.ages[0]]*self.kappa[t-self.years[0]])
        return value
    
    # POST: Estimates parameters alpha_x, beta_x and kappa_t
    #       by means of algorithm 2.1 provided in the bachelor thesis
    #       using tolerance tol (and the maximum number of cycles given)
    #       and computes the fitted log-mortality and mortality rates
    def doEstimateParameters(self, maxNumCycles=1000, tol=10**(-10)):
        counter = 0
        previousLikelihood = 0
        sumxNum = 0
        sumxDen = 0
        sumtNum = 0
        sumtDen = 0
        sumKappa = 0
        N = len(self.years) # number of years considered
        sumBeta = 0

        while counter < maxNumCycles:
            # Step 1: Updating alpha_x
            for x in self.ages:
                sumtNum = 0
                sumtDen = 0
                for t in self.years:
                    sumtNum += dxt(x,t,self.gender)
                    sumtNum -= Ext(x,t,self.gender) * np.exp(self.alpha[x-self.ages[0]] + self.beta[x-self.ages[0]]*self.kappa[t-self.years[0]])
                    sumtDen += Ext(x,t,self.gender) * np.exp(self.alpha[x-self.ages[0]] + self.beta[x-self.ages[0]]*self.kappa[t-self.years[0]])
                if sumtDen != 0:
                    self.alpha[x-self.ages[0]] += sumtNum / sumtDen
                else:
                    print(f"Attention: Division by zero at age {x} when updating parameter $\alpha_x$! The parameter value was not updated in the respective iteration step.")

            # Step 2: Updating kappa_t
            for t in self.years:
                sumxNum = 0
                sumxDen = 0
                for x in self.ages:
                    sumxNum = sumxNum + (dxt(x,t,self.gender) - Ext(x,t,self.gender)*np.exp(self.alpha[x-self.ages[0]] + self.beta[x-self.ages[0]]*self.kappa[t-self.years[0]])) * self.beta[x-self.ages[0]]
                    sumxDen += Ext(x,t,self.gender) * np.exp(self.alpha[x-self.ages[0]] + self.beta[x-self.ages[0]]*self.kappa[t-self.years[0]]) * (self.beta[x-self.ages[0]])**2
                if sumxDen != 0:
                    self.kappa[t-self.years[0]] += sumxNum / sumxDen
                else:
                    print(f"Attention: Division by zero in year {t} when updating parameter $\kappa_t$! The parameter value was not updated in the respective iteration step.")

            # Imposing constraint on kappa_t
            sumKappa = 0
            for t in self.years:
                sumKappa += self.kappa[t-self.years[0]]
            for t in self.years:
                self.kappa[t-self.years[0]] -= 1/N * sumKappa

            # Step 3: Updating beta_x
            for x in self.ages:
                sumtNum = 0
                sumtDen = 0
                for t in self.years:
                    sumtNum = sumtNum + (dxt(x,t,self.gender) - Ext(x,t,self.gender)*np.exp(self.alpha[x-self.ages[0]] + self.beta[x-self.ages[0]]*self.kappa[t-self.years[0]])) * self.kappa[t-self.years[0]]
                    sumtDen += Ext(x,t,self.gender) * np.exp(self.alpha[x-self.ages[0]] + self.beta[x-self.ages[0]]*self.kappa[t-self.years[0]]) * (self.kappa[t-self.years[0]])**2
                if sumtDen != 0:
                    self.beta[x-self.ages[0]] += sumtNum / sumtDen
                else:
                    print(f"Attention: Division by zero at age {x} when updating parameter $\beta_x$! The parameter value was not updated in the respective iteration step.")

            currentLikelihood = self.getLogLikelihood()
            if counter > 0 and (currentLikelihood - previousLikelihood) >= 0 and (currentLikelihood - previousLikelihood) < tol:
                break
            else:
                previousLikelihood = currentLikelihood
                counter += 1

        # Imposing constraint on beta_x
        for x in self.ages:
            sumBeta += self.beta[x-self.ages[0]]
        assert sumBeta != 0, r"The sum of all parameter values $\beta_x$ is equal to 0!"
        for x in self.ages:
            self.beta[x-self.ages[0]] /= sumBeta
        for t in self.years:
            self.kappa[t-self.years[0]] *= sumBeta

        if counter >= maxNumCycles:
            print(f"Parameters failed to be estimated in {counter} cycles using tolerance {tol}! Adjust the maximum number of cycles or the tolerance!")
        else:
            self.fitted = True
            print(f"Parameters were estimated successfully in {counter+1} cycles using tolerance {tol}!")
        
        # Computing fitted log-mortality and mortality rates
        for t in self.years:
            for x in self.ages:
                self.logMortality[x-self.ages[0],t-self.years[0]] = self.alpha[x-self.ages[0]] + self.beta[x-self.ages[0]]*self.kappa[t-self.years[0]]
                self.mortality[x-self.ages[0],t-self.years[0]] = np.exp(self.alpha[x-self.ages[0]] + self.beta[x-self.ages[0]]*self.kappa[t-self.years[0]])
                
        return
    
    # POST: Computes deviance residuals with the parameter values estimated
    def doComputeResiduals(self):
        # Estimating parameter values first unless already done
        if self.fitted != True:
            self.doEstimateParameters()
        for x in self.ages:
            for t in self.years:
                dEstimate = Ext(x,t,self.gender)*self.mortality[x-self.ages[0], t-self.years[0]]
                dActual = dxt(x,t,self.gender)
                sign = np.sign(dActual - dEstimate)
                if dEstimate != 0 and dActual != 0:
                    sqrt = np.sqrt(dActual * np.log(dActual/dEstimate) - (dActual - dEstimate))
                    self.residuals[x-self.ages[0],t-self.years[0]] = sign * sqrt
                elif dEstimate == 0 and dActual == 0:
                    self.residuals[x-self.ages[0],t-self.years[0]] = 0
                else:
                    self.residuals[x-self.ages[0],t-self.years[0]] = np.nan
                    print(f"Error: Division by zero or taking log(0) while computing the residual for age {x} and year {t}! The respective residual was set to NaN.")
        self.residualsComputed = True
        return
    
    # POST: Fits an appropriate ARIMA model to the time series kappa_t,
    #       produces forecasts thereof,
    #       computes forecasts of the log-mortality and mortality rate
    #       and determines confidence intervals for the respective quantities
    def doForecast(self):
        # Estimating parameter values first unless already done
        if self.fitted != True:
            self.doEstimateParameters()
        # Fitting an appropriate ARIMA model
        self.arima = pm.auto_arima(self.kappa,
                                   error_action='ignore', 
                                   trace=True, # displaying the search history
                                   suppress_warnings=True # no convergence warnings
                                  )

        # Forecasting kappa_t and returning (1-alpha) confidence intervals
        # according to approach 1 explained in subsection 2.3.1 of
        # the bachelor thesis
        self.kappaForecast, self.kappaConfIntForec = self.arima.predict(n_periods=len(self.yearsForecasted),
                                                                        return_conf_int=True,
                                                                        alpha=0.05)
        
        # Calculating forecasts of the log-mortality and mortality rate
        # and confidence intervals thereof according to approach 1
        # explained in subsection 2.3.1 of the bachelor thesis
        for i in range(len(self.yearsForecasted)):
            for j in range(len(self.ages)):
                self.logMortalityForecast[j,i] = self.alpha[j] + self.beta[j]*self.kappaForecast[i]
                self.mortalityForecast[j,i] = np.exp(self.alpha[j] + self.beta[j]*self.kappaForecast[i])
                if self.beta[j] >= 0:
                    self.logMortConfIntForec[j,i,0] = self.alpha[j] + self.beta[j]*self.kappaConfIntForec[i,0]
                    self.logMortConfIntForec[j,i,1] = self.alpha[j] + self.beta[j]*self.kappaConfIntForec[i,1]

                    self.mortConfIntForec[j,i,0] = np.exp(self.alpha[j] + self.beta[j]*self.kappaConfIntForec[i,0])
                    self.mortConfIntForec[j,i,1] = np.exp(self.alpha[j] + self.beta[j]*self.kappaConfIntForec[i,1])
                else:
                    self.logMortConfIntForec[j,i,1] = self.alpha[j] + self.beta[j]*self.kappaConfIntForec[i,0]
                    self.logMortConfIntForec[j,i,0] = self.alpha[j] + self.beta[j]*self.kappaConfIntForec[i,1]

                    self.mortConfIntForec[j,i,1] = np.exp(self.alpha[j] + self.beta[j]*self.kappaConfIntForec[i,0])
                    self.mortConfIntForec[j,i,0] = np.exp(self.alpha[j] + self.beta[j]*self.kappaConfIntForec[i,1])
        
        self.forecasted = True
        print("Forecasts were computed successfully!")
        return
    
    # POST: Performs the bootstrap method specified (wild or parametric)
    #       with "numOfSamples" samples from the respective Poisson distribution
    #       and uses level "percent" to obtain approximate confidence intervals
    def doBootstrap(self, numOfSamples, percent, wild=False):
        # Estimating parameter values first unless already done
        if self.fitted != True:
            self.doEstimateParameters()
        bootstrap = Bootstrap(self.ages[0], self.ages[-1],
                              self.years[0], self.years[-1],
                              self.gender, len(self.yearsForecasted),
                              numOfSamples, self.mortality)
        bootstrap.doSample(wild)
        bootstrap.doEstimateParametersAll()
        bootstrap.doComputeConfInt(percent)
        bootstrap.doForecastAll()
        bootstrap.doComputeConfIntForecasts(percent)
        
        # approximate confidence intervals for alpha_x, beta_x and kappa_t
        self.alphaConfInt = bootstrap.alphaConfInt
        self.betaConfInt = bootstrap.betaConfInt
        self.kappaConfInt = bootstrap.kappaConfInt
        
        # approximate confidence intervals for log-mortality and mortality rates
        self.logMortConfInt = bootstrap.logMortConfInt
        self.mortConfInt = bootstrap.mortConfInt
        
        # approximate confidence intervals for forecasts of kappa_t,
        # log-mortality and mortality rates
        # only accounting for parameter estimation uncertainty
        # (corresponding to approach 2 explained in subsection 2.3.1
        # of the bachelor thesis)
        self.kappaAppConfIntForec = bootstrap.kappaAppConfIntForec
        self.logMortAppConfIntForec = bootstrap.logMortAppConfIntForec
        self.mortAppConfIntForec = bootstrap.mortAppConfIntForec
        
        # approximate confidence intervals for forecasts of kappa_t,
        # log-mortality and mortality rates
        # accounting for both uncertainties:
        # the one in the parameter estimation and the one in the forecasting
        # (corresponding to the combined approach explained in subsection 2.3.1
        # of the bachelor thesis)
        self.kappaCompConfIntForec = bootstrap.kappaCompConfIntForec
        self.logMortCompConfIntForec = bootstrap.logMortCompConfIntForec
        self.mortCompConfIntForec = bootstrap.mortCompConfIntForec
        
        self.bootstrapped = True
        print("Bootstrapping was completed successfully!")
        
        return

In [ ]:
class Bootstrap:
    
    # PRE:  ageStart <= ageEnd and both in interval agesAvailable
    #       periodStart <= periodEnd and both in interval periodAvailable
    #       gender binary: 0 for female, 1 for male
    # POST: Initializes a bootstrap procedure with the attributes provided
    def __init__(self, ageStart, ageEnd, periodStart, periodEnd, gender,
                 numOfYearsToForecast, numOfSamples, fittedMortality):
        if ageEnd > agesAvailable[1]:
            print(f"For ages greater than {agesAvailable[1]}, only aggregate data including all ages greater and equal to {agesAvailable[1]} is available. Hence, the maximum age was set to {agesAvailable[1]}.")
            ageEnd = agesAvailable[1]
        assert ageEnd >= ageStart, "The first age considered must be below or equal to the last age considered!"
        assert ageStart >= agesAvailable[0], f"Data not available for ages below {agesAvailable[0]}!"
        self.ages = np.arange(ageStart, ageEnd+1) # ages considered
        
        assert periodEnd >= periodStart, "The first year considered must be earlier than or the same as the last year considered!"
        assert periodEnd <= periodAvailable[1] and periodStart >= periodAvailable[0], "Data not available for the period selected!"
        self.years = np.arange(periodStart, periodEnd+1) # years considered
        
        assert gender == 1 or gender == 0, "Gender must be binary!"
        self.gender = gender # gender considered: 0 for female, 1 for male
        
        # fitted mortality rates (based on actual data)
        self.initialMortality = fittedMortality
        
        self.N = numOfSamples # number of samples to be generated
        self.dxt = np.zeros((ageEnd-ageStart+1, periodEnd-periodStart+1,
                             numOfSamples)) # sampled death counts
        
        # estimated parameters alpha_x, beta_x and kappa_t
        self.alpha = np.zeros((ageEnd-ageStart+1, numOfSamples))
        self.beta = np.ones((ageEnd-ageStart+1, numOfSamples))
        self.kappa = np.zeros((periodEnd-periodStart+1, numOfSamples))
        
        # fitted log-mortality and mortality rates
        self.logMortality = np.zeros((ageEnd-ageStart+1,
                                      periodEnd-periodStart+1, numOfSamples))
        self.mortality = np.zeros((ageEnd-ageStart+1,
                                   periodEnd-periodStart+1, numOfSamples))
        
        # approximate confidence intervals for alpha_x, beta_x and kappa_t
        self.alphaConfInt = np.zeros((ageEnd-ageStart+1, 2))
        self.betaConfInt = np.zeros((ageEnd-ageStart+1, 2))
        self.kappaConfInt = np.zeros((periodEnd-periodStart+1, 2))
        
        # approximate confidence intervals for log-mortality and
        # mortality rates
        self.logMortConfInt = np.zeros((ageEnd-ageStart+1,
                                        periodEnd-periodStart+1, 2))
        self.mortConfInt = np.zeros((ageEnd-ageStart+1,
                                     periodEnd-periodStart+1, 2))
        
        # Forecasts are produced for the following years
        self.yearsForecasted = np.arange(periodEnd+1,
                                         periodEnd+numOfYearsToForecast+1)
        
        # forecasts of kappa_t and confidence intervals thereof
        self.kappaForecast = np.zeros((numOfYearsToForecast, numOfSamples))
        self.kappaConfIntForec = np.zeros((numOfYearsToForecast, 2,
                                           numOfSamples))
        
        # forecasts of log-mortality and mortality rate
        # and confidence intervals thereof
        self.logMortalityForecast = np.zeros((ageEnd-ageStart+1,
                                              numOfYearsToForecast,
                                              numOfSamples))
        self.logMortConfIntForec = np.zeros((ageEnd-ageStart+1,
                                             numOfYearsToForecast, 2,
                                             numOfSamples))
        self.mortalityForecast = np.zeros((ageEnd-ageStart+1,
                                           numOfYearsToForecast,
                                           numOfSamples))
        self.mortConfIntForec = np.zeros((ageEnd-ageStart+1,
                                          numOfYearsToForecast, 2,
                                          numOfSamples))
        
        # approximate confidence intervals for kappa_t, log-mortality
        # and mortality rates according to approach 2 in subsection 2.3.1
        # of the bachelor thesis
        self.kappaAppConfIntForec = np.zeros((numOfYearsToForecast, 2))
        self.logMortAppConfIntForec = np.zeros((ageEnd-ageStart+1,
                                                numOfYearsToForecast, 2))
        self.mortAppConfIntForec = np.zeros((ageEnd-ageStart+1,
                                             numOfYearsToForecast, 2))
        
        # approximate confidence intervals for kappa_t, log-mortality
        # and mortality rates according to the combined approach in 
        # subsection 2.3.1 of the bachelor thesis
        self.kappaCompConfIntForec = np.zeros((numOfYearsToForecast, 2))
        self.logMortCompConfIntForec = np.zeros((ageEnd-ageStart+1,
                                                 numOfYearsToForecast, 2))
        self.mortCompConfIntForec = np.zeros((ageEnd-ageStart+1,
                                              numOfYearsToForecast, 2))
        
        self.sampled = False # Samples already generated?
        self.fitted = False # Model already fitted?
        self.forecasted = False # Forecasts already produced?
    
    # POST: Samples death counts from a Poisson distribution with mean:
    #       - d_x(t) if wild == True
    #       - E_x(t) * np.exp(alpha_x + beta_x * kappa_t) if wild == False,
    #         where the parameter values are the ones initially obtained from
    #         fitting the Lee-Carter model with the actual numbers of deaths
    def doSample(self, wild=False):
        if wild == True:
            for x in self.ages:
                for t in self.years:
                    rng = np.random.default_rng()
                    self.dxt[x-self.ages[0], t-self.years[0],:] = rng.poisson(dxt(x,t,self.gender), self.N)
        else:
            for x in self.ages:
                for t in self.years:
                    rng = np.random.default_rng()
                    self.dxt[x-self.ages[0], t-self.years[0],:] = rng.poisson(Ext(x,t,self.gender) * self.initialMortality[x-self.ages[0], t-self.years[0]], self.N)
        self.sampled = True
        return
    
    # PRE:  0 <= sampleNum <= self.N
    # POST: Returns value of log-likelihood function without the constant term
    #       for the specified sample
    def getLogLikelihood(self, sampleNum):
        assert sampleNum >= 0 and sampleNum < self.N, "Sample number is out of range!"
        value = 0
        for t in self.years:
            for x in self.ages:
                value += self.dxt[x-self.ages[0], t-self.years[0], sampleNum] * (self.alpha[x-self.ages[0],sampleNum] + self.beta[x-self.ages[0],sampleNum]*self.kappa[t-self.years[0],sampleNum])
                value -= Ext(x,t,self.gender) * np.exp(self.alpha[x-self.ages[0],sampleNum] + self.beta[x-self.ages[0],sampleNum]*self.kappa[t-self.years[0],sampleNum])
        return value
    
    # PRE:  0 <= sampleNum < self.N
    # POST: Estimates parameters alpha_x, beta_x and kappa_t
    #       by means of algorithm 2.1 provided in the bachelor thesis
    #       using tolerance tol (and the maximum number of cycles given)
    #       and computes the fitted log-mortality and mortality rates
    #       for the specified sample
    def doEstimateParameters(self, sampleNum, maxNumCycles=1000, tol=10**(-10)):
        assert sampleNum >= 0 and sampleNum < self.N, "Sample number is out of range!"
        # Sampling first unless already done
        if self.sampled != True:
            self.doSample()
        counter = 0
        successful = True
        previousLikelihood = 0
        sumxNum = 0
        sumxDen = 0
        sumtNum = 0
        sumtDen = 0
        sumKappa = 0
        N = len(self.years) # number of years considered
        sumBeta = 0

        while counter < maxNumCycles:
            # Step 1: Updating alpha_x
            for x in self.ages:
                sumtNum = 0
                sumtDen = 0
                for t in self.years:
                    sumtNum += self.dxt[x-self.ages[0], t-self.years[0], sampleNum]
                    sumtNum -= Ext(x,t,self.gender) * np.exp(self.alpha[x-self.ages[0],sampleNum] + self.beta[x-self.ages[0],sampleNum]*self.kappa[t-self.years[0],sampleNum])
                    sumtDen += Ext(x,t,self.gender) * np.exp(self.alpha[x-self.ages[0],sampleNum] + self.beta[x-self.ages[0],sampleNum]*self.kappa[t-self.years[0],sampleNum])
                if sumtDen != 0:
                    self.alpha[x-self.ages[0],sampleNum] += sumtNum / sumtDen
                else:
                    print(f"Attention: Division by zero at age {x} when updating parameter $\alpha_x$ for sample number {sampleNum}! The parameter value was not updated in the respective iteration step.")
                    
            # Step 2: Updating kappa_t
            for t in self.years:
                sumxNum = 0
                sumxDen = 0
                for x in self.ages:
                    sumxNum = sumxNum + (self.dxt[x-self.ages[0], t-self.years[0], sampleNum] - Ext(x,t,self.gender)*np.exp(self.alpha[x-self.ages[0],sampleNum] + self.beta[x-self.ages[0],sampleNum]*self.kappa[t-self.years[0],sampleNum])) * self.beta[x-self.ages[0],sampleNum]
                    sumxDen += Ext(x,t,self.gender) * np.exp(self.alpha[x-self.ages[0],sampleNum] + self.beta[x-self.ages[0],sampleNum]*self.kappa[t-self.years[0],sampleNum]) * (self.beta[x-self.ages[0],sampleNum])**2
                if sumxDen != 0:
                    self.kappa[t-self.years[0],sampleNum] += sumxNum / sumxDen
                else:
                    print(f"Attention: Division by zero in year {t} when updating parameter $\kappa_t$ for sample number {sampleNum}! The parameter value was not updated in the respective iteration step.")

            # Imposing constraint on kappa_t
            sumKappa = 0
            for t in self.years:
                sumKappa += self.kappa[t-self.years[0],sampleNum]
            for t in self.years:
                self.kappa[t-self.years[0],sampleNum] -= 1/N * sumKappa

            # Step 3: Updating beta_x
            for x in self.ages:
                sumtNum = 0
                sumtDen = 0
                for t in self.years:
                    sumtNum = sumtNum + (self.dxt[x-self.ages[0], t-self.years[0], sampleNum] - Ext(x,t,self.gender)*np.exp(self.alpha[x-self.ages[0],sampleNum] + self.beta[x-self.ages[0],sampleNum]*self.kappa[t-self.years[0],sampleNum])) * self.kappa[t-self.years[0],sampleNum]
                    sumtDen += Ext(x,t,self.gender) * np.exp(self.alpha[x-self.ages[0],sampleNum] + self.beta[x-self.ages[0],sampleNum]*self.kappa[t-self.years[0],sampleNum]) * (self.kappa[t-self.years[0],sampleNum])**2
                if sumtDen != 0:
                    self.beta[x-self.ages[0],sampleNum] += sumtNum / sumtDen
                else:
                    print(f"Attention: Division by zero at age {x} when updating parameter $\beta_x$ for sample number {sampleNum}! The parameter value was not updated in the respective iteration step.")

            currentLikelihood = self.getLogLikelihood(sampleNum)
            if counter > 0 and (currentLikelihood - previousLikelihood) >= 0 and (currentLikelihood - previousLikelihood) < tol:
                break
            else:
                previousLikelihood = currentLikelihood
                counter += 1

        # Imposing constraint on beta_x
        for x in self.ages:
            sumBeta += self.beta[x-self.ages[0],sampleNum]
        assert sumBeta != 0, f"The sum of all parameter values $\beta_x$ is equal to 0 for sample number {sampleNum}!"
        for x in self.ages:
            self.beta[x-self.ages[0],sampleNum] /= sumBeta
        for t in self.years:
            self.kappa[t-self.years[0],sampleNum] *= sumBeta

        if counter >= maxNumCycles:
            print(f"Parameters for sample number {sampleNum} failed to be estimated in {counter} cycles using tolerance {tol}! Adjust the maximum number of cycles or the tolerance!")
            successful = False
        
        for t in self.years:
            for x in self.ages:
                self.logMortality[x-self.ages[0],t-self.years[0],sampleNum] = self.alpha[x-self.ages[0],sampleNum] + self.beta[x-self.ages[0],sampleNum]*self.kappa[t-self.years[0],sampleNum]
                self.mortality[x-self.ages[0],t-self.years[0],sampleNum] = np.exp(self.alpha[x-self.ages[0],sampleNum] + self.beta[x-self.ages[0],sampleNum]*self.kappa[t-self.years[0],sampleNum])
                
        return successful
    
    # POST: Estimates parameters alpha_x, beta_x and kappa_t
    #       by means of algorithm 2.1 provided in the bachelor thesis
    #       for all samples
    def doEstimateParametersAll(self):
        # Sampling first unless already done
        if self.sampled != True:
            self.doSample()
        successfulAll = True
        for i in range(self.N):
            successful = self.doEstimateParameters(i)
            if successful == False:
                successfulAll = False
        self.fitted = True
        if successfulAll = True:
            print(f"Parameters were estimated successfully!")
        else:
            print(f"Parameters failed to be estimated for the samples indicated above! For all others, parameters were estimated successfully!")
        return
    
    # PRE:  0 <= percent <= 100
    # POST: Computes approximate confidence intervals for alpha_x, beta_x,
    #       kappa_t, fitted log-mortality and mortality rates
    #       at the level specified by percent
    def doComputeConfInt(self, percent):
        assert percent >= 0 and percent <= 100, "The percentage value for the confidence intervals must be between 0 and 100!"
        # Estimating parameters first unless already done
        if self.fitted != True:
            self.doEstimateParametersAll()
        tailsPercentage = ((100 - percent)/2) / 100
        tails = int(np.floor(tailsPercentage * self.N))
        alpha = self.alpha
        beta = self.beta
        kappa = self.kappa
        logMort = self.logMortality
        mort = self.mortality
        for x in self.ages:
            alpha[x-self.ages[0],:] = np.sort(self.alpha[x-self.ages[0],:])
            self.alphaConfInt[x-self.ages[0],0] = alpha[x-self.ages[0],tails]
            self.alphaConfInt[x-self.ages[0],1] = alpha[x-self.ages[0],-(tails+1)]
            beta[x-self.ages[0],:] = np.sort(self.beta[x-self.ages[0],:])
            self.betaConfInt[x-self.ages[0],0] = beta[x-self.ages[0],tails]
            self.betaConfInt[x-self.ages[0],1] = beta[x-self.ages[0],-(tails+1)]
        for t in self.years:
            kappa[t-self.years[0],:] = np.sort(self.kappa[t-self.years[0],:])
            self.kappaConfInt[t-self.years[0],0] = kappa[t-self.years[0],tails]
            self.kappaConfInt[t-self.years[0],1] = kappa[t-self.years[0],-(tails+1)]
        for x in self.ages:
            for t in self.years:
                logMort[x-self.ages[0],t-self.years[0],:] = np.sort(self.logMortality[x-self.ages[0],t-self.years[0],:])
                self.logMortConfInt[x-self.ages[0],t-self.years[0],0] = logMort[x-self.ages[0],t-self.years[0],tails]
                self.logMortConfInt[x-self.ages[0],t-self.years[0],1] = logMort[x-self.ages[0],t-self.years[0],-(tails+1)]
                mort[x-self.ages[0],t-self.years[0],:] = np.sort(self.mortality[x-self.ages[0],t-self.years[0],:])
                self.mortConfInt[x-self.ages[0],t-self.years[0],0] = mort[x-self.ages[0],t-self.years[0],tails]
                self.mortConfInt[x-self.ages[0],t-self.years[0],1] = mort[x-self.ages[0],t-self.years[0],-(tails+1)]
        actualPercent = 100 - 2*(tails/self.N)*100
        print(f"Approximate {actualPercent}% confidence intervals were computed successfully!")
        return         
    
    # PRE:  0 <= sampleNum < self.N
    # POST: Fits an appropriate ARIMA model to the time series kappa_t,
    #       produces forecasts thereof,
    #       computes forecasts of the log-mortality and mortality rate
    #       and determines confidence intervals for the respective quantities
    #       for the specified sample
    def doForecast(self, sampleNum):
        assert sampleNum >= 0 and sampleNum < self.N, "Sample number is out of range!"
        # Estimating parameters first unless already done
        if self.fitted != True:
            self.doEstimateParametersAll()
        # Fitting an appropriate ARIMA model
        arima = pm.auto_arima(self.kappa[:,sampleNum],
                              error_action='ignore',
                              trace=False, # not displaying the search history
                              suppress_warnings=True # no convergence warnings
                             )

        # Forecasting kappa_t and returning (1-alpha) confidence intervals
        self.kappaForecast[:,sampleNum], self.kappaConfIntForec[:,:,sampleNum] = arima.predict(n_periods=len(self.yearsForecasted),
                                                                                               return_conf_int=True,
                                                                                               alpha=0.05)
        
        # Calculating forecasts of the log-mortality and mortality rate
        # and confidence intervals thereof
        for i in range(len(self.yearsForecasted)):
            for j in range(len(self.ages)):
                self.logMortalityForecast[j,i,sampleNum] = self.alpha[j,sampleNum] + self.beta[j,sampleNum]*self.kappaForecast[i,sampleNum]
                self.mortalityForecast[j,i,sampleNum] = np.exp(self.alpha[j,sampleNum] + self.beta[j,sampleNum]*self.kappaForecast[i,sampleNum])
                if self.beta[j,sampleNum] >= 0:
                    self.logMortConfIntForec[j,i,0,sampleNum] = self.alpha[j,sampleNum] + self.beta[j,sampleNum]*self.kappaConfIntForec[i,0,sampleNum]
                    self.logMortConfIntForec[j,i,1,sampleNum] = self.alpha[j,sampleNum] + self.beta[j,sampleNum]*self.kappaConfIntForec[i,1,sampleNum]

                    self.mortConfIntForec[j,i,0,sampleNum] = np.exp(self.alpha[j,sampleNum] + self.beta[j,sampleNum]*self.kappaConfIntForec[i,0,sampleNum])
                    self.mortConfIntForec[j,i,1,sampleNum] = np.exp(self.alpha[j,sampleNum] + self.beta[j,sampleNum]*self.kappaConfIntForec[i,1,sampleNum])
                else:
                    self.logMortConfIntForec[j,i,1,sampleNum] = self.alpha[j,sampleNum] + self.beta[j,sampleNum]*self.kappaConfIntForec[i,0,sampleNum]
                    self.logMortConfIntForec[j,i,0,sampleNum] = self.alpha[j,sampleNum] + self.beta[j,sampleNum]*self.kappaConfIntForec[i,1,sampleNum]

                    self.mortConfIntForec[j,i,1,sampleNum] = np.exp(self.alpha[j,sampleNum] + self.beta[j,sampleNum]*self.kappaConfIntForec[i,0,sampleNum])
                    self.mortConfIntForec[j,i,0,sampleNum] = np.exp(self.alpha[j,sampleNum] + self.beta[j,sampleNum]*self.kappaConfIntForec[i,1,sampleNum])
        
        return

    # POST: Fits an appropriate ARIMA model to the time series kappa_t,
    #       produces forecasts thereof,
    #       computes forecasts of the log-mortality and mortality rate
    #       and determines confidence intervals for the respective quantities
    #       for all samples
    def doForecastAll(self):
        # Estimating parameters first unless already done
        if self.fitted != True:
            self.doEstimateParametersAll()
        for i in range(self.N):
            self.doForecast(i)
        self.forecasted = True
        print("Forecasts were computed successfully!")
        return
    
    # PRE:  0 <= percent <= 100
    # POST: Computes approximate confidence intervals for the forecasts of
    #       kappa_t, the log-mortality and mortality rate
    #       at the level specified by percent;
    #       on the one hand, by accounting only for parameter estimation
    #       uncertainty (corresponding to approach 2 in
    #       subsection 2.3.1 of the bachelor thesis); 
    #       on the other hand, by also accounting for the uncertainty
    #       in the forecasts of kappa_t (corresponding to the combined
    #       approach in subsection 2.3.1 of the bachelor thesis)
    def doComputeConfIntForecasts(self, percent):
        assert percent >= 0 and percent <= 100, "The percentage value for the confidence intervals must be between 0 and 100!"
        # Forecasting first unless already done
        if self.forecasted != True:
            self.doForecastAll()
        tailsPercentage = ((100 - percent)/2) / 100
        tails = int(np.floor(tailsPercentage * self.N))
        kappa = self.kappaForecast
        orderKappa = np.zeros(len(self.yearsForecasted))
        kappaInt = np.zeros(self.N - 2*tails)
        logMort = self.logMortalityForecast
        orderLogMort = np.zeros(len(self.yearsForecasted))
        logMortInt = np.zeros(self.N - 2*tails)
        mort = self.mortalityForecast
        orderMort = np.zeros(len(self.yearsForecasted))
        mortInt = np.zeros(self.N - 2*tails)
        for t in self.yearsForecasted:
            kappa[t-self.yearsForecasted[0],:] = np.sort(self.kappaForecast[t-self.yearsForecasted[0],:])
            orderKappa = np.argsort(self.kappaForecast[t-self.yearsForecasted[0],:])
            
            # approach 2
            self.kappaAppConfIntForec[t-self.yearsForecasted[0],0] = kappa[t-self.yearsForecasted[0],tails]
            self.kappaAppConfIntForec[t-self.yearsForecasted[0],1] = kappa[t-self.yearsForecasted[0],-(tails+1)]
            
            # combined approach
            self.kappaCompConfIntForec[t-self.yearsForecasted[0],0] = np.sort(self.kappaConfIntForec[t-self.yearsForecasted[0],0,orderKappa[tails:-(tails+1)]])[0]
            self.kappaCompConfIntForec[t-self.yearsForecasted[0],1] = np.sort(self.kappaConfIntForec[t-self.yearsForecasted[0],1,orderKappa[tails:-(tails+1)]])[-1]
        for x in self.ages:
            for t in self.yearsForecasted:
                logMort[x-self.ages[0],t-self.yearsForecasted[0],:] = np.sort(self.logMortalityForecast[x-self.ages[0],t-self.yearsForecasted[0],:])
                orderLogMort = np.argsort(self.logMortalityForecast[x-self.ages[0],t-self.yearsForecasted[0],:])
                
                # approach 2
                self.logMortAppConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],0] = logMort[x-self.ages[0],t-self.yearsForecasted[0],tails]
                self.logMortAppConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],1] = logMort[x-self.ages[0],t-self.yearsForecasted[0],-(tails+1)]
                
                # combined approach
                self.logMortCompConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],0] = np.sort(self.logMortConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],0,orderLogMort[tails:-(tails+1)]])[0]
                self.logMortCompConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],1] = np.sort(self.logMortConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],1,orderLogMort[tails:-(tails+1)]])[-1]
                
                mort[x-self.ages[0],t-self.yearsForecasted[0],:] = np.sort(self.mortalityForecast[x-self.ages[0],t-self.yearsForecasted[0],:])
                orderMort = np.argsort(self.mortalityForecast[x-self.ages[0],t-self.yearsForecasted[0],:])
                
                # approach 2
                self.mortAppConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],0] = mort[x-self.ages[0],t-self.yearsForecasted[0],tails]
                self.mortAppConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],1] = mort[x-self.ages[0],t-self.yearsForecasted[0],-(tails+1)]
                
                # combined approach
                self.mortCompConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],0] = np.sort(self.mortConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],0,orderMort[tails:-(tails+1)]])[0]
                self.mortCompConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],1] = np.sort(self.mortConfIntForec[x-self.ages[0],t-self.yearsForecasted[0],1,orderMort[tails:-(tails+1)]])[-1]
        
        actualPercent = 100 - 2*(tails/self.N)*100
        print(f"Approximate {actualPercent}% confidence intervals were computed successfully!")
        return

## Applying the model

### Initializing the model

In [ ]:
ageStartFemale = 0 # first age under consideration
ageEndFemale = 100 # last age under consideration
periodStartFemale = 1935 # first calendar year under consideration
periodEndFemale = 2000 # last calendar year under consideration
numOfYearsToForecastFemale = 20 # number of years to be forecasted

# Initializing model for females
modelFemale = Lee_Carter(ageStartFemale, ageEndFemale,
                         periodStartFemale, periodEndFemale, 0,
                         numOfYearsToForecastFemale)

In [ ]:
ageStartMale = 0 # first age under consideration
ageEndMale = 100 # last age under consideration
periodStartMale = 1915 # first calendar year under consideration
periodEndMale = 2000 # last calendar year under consideration
numOfYearsToForecastMale = 20 # number of years to be forecasted

# Initializing model for males
modelMale = Lee_Carter(ageStartMale, ageEndMale,
                       periodStartMale, periodEndMale, 1,
                       numOfYearsToForecastMale)

### Fitting the model to past data

In [ ]:
# Fitting model for females
modelFemale.doEstimateParameters()

In [ ]:
# Fitting model for males
modelMale.doEstimateParameters()

### Computing the deviance residuals

In [ ]:
# Computing deviance residuals for females
modelFemale.doComputeResiduals()

In [ ]:
# Computing deviance residuals for males
modelMale.doComputeResiduals()

### Forecasting the time trend and the mortality rates

In [ ]:
# Forecasting kappa_t and mortality rates for females
modelFemale.doForecast()

In [ ]:
# Forecasting kappa_t and mortality rates for males
modelMale.doForecast()

### Performing the parametric bootstrap and calculating the approximate confidence intervals

In [ ]:
# Bootstrapping model for females with 1000 samples (parametric bootstrap)
# Attention: This computation may take pretty long!
modelFemale.doBootstrap(1000, 95)

In [ ]:
# Bootstrapping model for males with 1000 samples (parametric bootstrap)
# Attention: This computation may take pretty long!
modelMale.doBootstrap(1000, 95)